<h1>Amazon Dataset Processing</h1>

<h3>1. Function to return: Author, ISBN-10, ISBN-13, platform, price, category1, category2, description</h3>

Update From <b>Akshay, Zhuo</b> - 
<font color='red'>Information available from Kaggle:</font>

<li>ISBN 10</li>
<li>Author</li>
<li>Platform</li>

<b>Information Being scraped:</b>

<li>Description</li>
<li>Price</li>

Categories aren't available for all books. We might have to skip this for the moment.

<h3>Read data from amazon_com_extras. </h3>

In [1]:
import pandas as pd
import random 
import json
from bs4 import BeautifulSoup as bs
import numpy as np
import requests
import time

my_csvfile='./file2_lily_from20001to23000.csv' #please modify filename to your own need
start_entry=20001
end_entry=23000 #akshay already did 1~1000 entry, edit yours accordingly 

path_kaggle = './amazon_com_extras.csv' #download from https://www.kaggle.com/ucffool/amazon-sales-rank-data-for-print-and-kindle-books/version/3

with open(path_kaggle):
    #Added fields since line 808 in csv has more than 6 cols and needs to be told to ignore extra data.
    
    fields=['ASIN','GROUP','FORMAT','TITLE','AUTHOR','PUBLISHER']
    df_kaggle = pd.read_csv(path_kaggle, encoding='latin1', dtype=str,usecols=fields)

In [2]:
Df_read = df_kaggle.iloc[start_entry:end_entry+1]

<h4>Note: Amazon will identify and block you as a bot, please exercise caution while pulling data. Use user agents and throttling to prevent sending multilpe requests</h4>

User Agent Code - Refer https://www.scrapehero.com/how-to-fake-and-rotate-user-agents-using-python-3/

Updated the following :

<li>Throttling</li>
<li>Saving data to a single dataframe</li>

<h3> 1. Pull data from the Kaggle Dataset. We'll be using ISBN 10 to scrape book descriptions</h3>

In [3]:
import requests
import numpy as np

def amazon_description(book_metadata):
    
    import random
    import time
    
    #New columns that will be added to book_metadata
    book_descriptions=list()
    book_price=list()
    book_rating=list()
    
    user_agent_list = [
           #Chrome
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
            'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
            'Mozilla/5.0 (Windows NT 5.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
            'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
            'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
            'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
            'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
            'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
            #Firefox
            'Mozilla/4.0 (compatible; MSIE 9.0; Windows NT 6.1)',
            'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko',
            'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0)',
            'Mozilla/5.0 (Windows NT 6.1; Trident/7.0; rv:11.0) like Gecko',
            'Mozilla/5.0 (Windows NT 6.2; WOW64; Trident/7.0; rv:11.0) like Gecko',
            'Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko',
            'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.0; Trident/5.0)',
            'Mozilla/5.0 (Windows NT 6.3; WOW64; Trident/7.0; rv:11.0) like Gecko',
            'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0)',
            'Mozilla/5.0 (Windows NT 6.1; Win64; x64; Trident/7.0; rv:11.0) like Gecko',
            'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; WOW64; Trident/6.0)',
            'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; Trident/6.0)',
            'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0; .NET CLR 2.0.50727; .NET CLR 3.0.4506.2152; .NET CLR 3.5.30729)'
        ]
    
    
    book_metadata['Description']=''
    book_metadata['Price']=''
    book_metadata['Rating']=''
    for i in range(len(book_metadata)):
        
        #User Agents to prevent bot detection
        user_agent=random.choice(user_agent_list)
        book_isbn=book_metadata['ASIN'].iloc[i]
        
        print(book_isbn)
        base_url = "https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-keywords="
        url=base_url+book_isbn
        print(url)
        headers={'User-Agent':user_agent}
        book_response=requests.get(url,headers=headers)
        if book_response.status_code != 200:
            print("Getting Book Details Failed")
            return None
        else:
            result_page=bs(book_response.content,'lxml')
            all_book_a_tags=result_page.find("a",class_="a-link-normal s-access-detail-page s-color-twister-title-link a-text-normal")
            try:
                book_url=all_book_a_tags.get("href")
                description,price,rating=get_book_description(book_url,user_agent_list)
            except: 
                continue
            print(description,price,rating)

        book_metadata['Description'].iloc[i]=description
        book_metadata['Price'].iloc[i]=price
        book_metadata['Rating'].iloc[i]=rating
        book_metadata.to_csv(my_csvfile,encoding='utf-8') #save data after each query

    
    return book_metadata
    



<h3> Write code to pull the following - </h3>
 
<h4> 
<li> Description </li>
<li> Rating </li>
<li> Price </li>

    
</h4>    

<h3> Ans save data at the same time to the local file  </h3>

In [4]:
def get_book_description(url,user_agent_list):
    book_description=''
    book_rating=''
    book_price=''
    print(url)
    #User Agents to prevent bot detection
    user_agent = random.choice(user_agent_list)
    headers = {'User-Agent': user_agent}
    
    #Throttling to spoof the server into believe these are generated by users
    time.sleep(7)
    
    book_description_response=requests.get(url,headers=headers)
    if book_description_response.status_code!=200:
        print("Error in scraping description")
        return None
    else:
        book_page=bs(book_description_response.content,"lxml")
        a1=book_page.find('div',{'id':'a-page'})
        a2=a1.find('div',class_='book en_US').find('div',class_='a-container').find('div',{'id':'centerCol'})
        try:
            book_description=a2.find('div',{'id':'bookDescription_feature_div'}).find('noscript').find('div').get_text()
        except:
            book_description=np.NAN
        try:
            book_rating=a2.find('div',{'id':'averageCustomerReviews_feature_div'}).find('span',class_='a-declarative').find('span',{'id':'acrPopover'}).find('span',class_='a-declarative').find('a').find('i').find('span').get_text().split()[0]    
        except:
            book_rating=np.NAN
        try:
            book_price=a2.find('div',{'id':'MediaMatrix'}).find('div').find('div',{'id':'tmmSwatches'}).find('ul').find('li',class_='swatchElement selected').find('span').find('span').find('span').find('span',class_='a-color-base').find('span').get_text().strip()
        except:
            book_price=np.NAN
        return book_description,book_price,book_rating

In [5]:
df_final=amazon_description(Df_read) 

/Users/LovelyCheuk/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/LovelyCheuk/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/LovelyCheuk/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/

B0082CBQHO
https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-keywords=B0082CBQHO
https://www.amazon.com/Ring-Promise-Devon-Rhodes-ebook/dp/B0082CBQHO
B00OFIBGQU
https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-keywords=B00OFIBGQU
https://www.amazon.com/Catch-Rabbit-Sean-Denton-Book-ebook/dp/B00OFIBGQU
B01CZCW3LY
https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-keywords=B01CZCW3LY
B073FKW2RF
https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-keywords=B073FKW2RF
B073RTVMTR
https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-keywords=B073RTVMTR
B009I4QR6K
https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-keywords=B009I4QR6K
B019967OWC
https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-keywords=B019967OWC
https://www.amazon.com/Maverick-Meets-M

https://www.amazon.com/Hearts-Other-Stories-Pembroke-Steel-ebook/dp/B00T0QRREM
B01BK564BC
https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-keywords=B01BK564BC
B079HT3FL2
https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-keywords=B079HT3FL2
https://www.amazon.com/Born-Embers-Phoenix-Rising-Book-ebook/dp/B079HT3FL2
B00QHMGQE4
https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-keywords=B00QHMGQE4
https://www.amazon.com/Fake-Till-You-Make-Investigates-ebook/dp/B00QHMGQE4
B00LLH4KC0
https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-keywords=B00LLH4KC0
B00LJ05DOS
https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-keywords=B00LJ05DOS
B01KN64A5Q
https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-keywords=B01KN64A5Q
B07CF7CXHM
https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3

B074G4LXBH
https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-keywords=B074G4LXBH
B008325XI6
https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-keywords=B008325XI6
B00JHNAWQS
https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-keywords=B00JHNAWQS
B013MOA4M4
https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-keywords=B013MOA4M4
B0071MFSES
https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-keywords=B0071MFSES
B0756VRXFC
https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-keywords=B0756VRXFC
B00NF6LU4Q
https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-keywords=B00NF6LU4Q
https://www.amazon.com/Essential-Oils-Guide-Aromatherapy-Rockefellers-ebook/dp/B00NF6LU4Q
B0031U06IM
https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&

B07BHHP4LS
https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-keywords=B07BHHP4LS
B075FX2P7B
https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-keywords=B075FX2P7B
https://www.amazon.com/Marked-Power-Book-1-ebook/dp/B075FX2P7B
B074KQ4BZY
https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-keywords=B074KQ4BZY
B01MXYEK83
https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-keywords=B01MXYEK83
B073YM98YS
https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-keywords=B073YM98YS
https://www.amazon.com/Armor-God-Understanding-Assault-Innocence-ebook/dp/B073YM98YS
B01MF9VW2U
https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-keywords=B01MF9VW2U
B00JU9A1ZQ
https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-keywords=B00JU9A1ZQ
https://www.amazon.com/Karma-Bite

...

B0123FHRU0
https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-keywords=B0123FHRU0
B003J359B0
https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-keywords=B003J359B0
B0716BK3Z8
https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-keywords=B0716BK3Z8
https://www.amazon.com/Altitude-Niel-Bushnell-ebook/dp/B0716BK3Z8
B01HC83I1U
https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-keywords=B01HC83I1U
B01MF849VN
https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-keywords=B01MF849VN
https://www.amazon.com/Surrogate-Episode-hotwife-exhibitionist-cuckold-ebook/dp/B01MF849VN
B00LRIUALS
https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-keywords=B00LRIUALS
B079D5FBVS
https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-keywords=B079D5FBVS
B07C94VCBY
https://www.a

https://www.amazon.com/Deadly-Jewels-Novel-Jeannette-Beauvoir-ebook/dp/B0140MQVX8
B00PXHHL4I
https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-keywords=B00PXHHL4I
B00MQGOF2A
https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-keywords=B00MQGOF2A
B071WKP8FH
https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-keywords=B071WKP8FH
B01CTDFLDG
https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-keywords=B01CTDFLDG
B074J4SHHY
https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-keywords=B074J4SHHY
B01EW54AG8
https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-keywords=B01EW54AG8
B01D4WO2MC
https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-keywords=B01D4WO2MC
B00QZD9H5K
https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-ke

B012LMU4QO
https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-keywords=B012LMU4QO
https://www.amazon.com/Box-Jumper-Novella-Lisa-Mannetti-ebook/dp/B012LMU4QO
B01M5LIWLO
https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-keywords=B01M5LIWLO
B013COKCZ8
https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-keywords=B013COKCZ8
B012LWZJ1O
https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-keywords=B012LWZJ1O
https://www.amazon.com/Cover-Girl-Nic-Tatano-ebook/dp/B012LWZJ1O
B07B29CYLX
https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-keywords=B07B29CYLX
https://www.amazon.com/Crafters-Passion-Kris-Schnee-ebook/dp/B07B29CYLX
B01MQKK4HG
https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-keywords=B01MQKK4HG
B076927NW2
https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl

https://www.amazon.com/Find-Him-Kill-James-Hogarth-ebook/dp/B01D3O5VVM
B00P2S4YKC
https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-keywords=B00P2S4YKC
B073TNT5XX
https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-keywords=B073TNT5XX
B073YHYQWB
https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-keywords=B073YHYQWB
https://www.amazon.com/Bossy-Billionaire-Office-Romance-Chances-ebook/dp/B073YHYQWB
B01DLE0TIO
https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-keywords=B01DLE0TIO
B01DHUX24K
https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-keywords=B01DHUX24K
B071JRZTS4
https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-keywords=B071JRZTS4
https://www.amazon.com/Lightning-Strikes-Twice-Heart-Hero-ebook/dp/B071JRZTS4
B01B53SF8E
https://www.amazon.com/s/ref=nb_sb_noss?url=search-ali

KeyboardInterrupt: 